In [59]:
import warnings
import pandas 
pandas.set_option('mode.use_inf_as_na', True)

warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")

# This is required to catch warnings when the multiprocessing module is used
import os

os.environ["PYTHONWARNINGS"] = "ignore"
# import pertpy as pt
import scanpy as sc

import pertpy as pt
adata = pt.dt.kang_2018()
from torch import tensor
import torch, numpy as np, pandas as pd
np.set_printoptions(linewidth=140)
torch.set_printoptions(linewidth=140, sci_mode=False, edgeitems=7)
pd.set_option('display.width', 140)
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

In [60]:
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata)

hv_genes = (list(adata.var[adata.var['highly_variable'] == True].index))
normal_genes = (list(adata.var_names))

hv_columns = [i for i,val in enumerate(normal_genes) if val in hv_genes]
len(hv_columns)

1293

In [61]:
#preprocess
#batchcontrol
#clustering
#cell type annotation
#embedding
#reverse transcriptome
#differential expression

#rna velocity -> predicts the future state 
#predict the probability of unseen perturbations
#test: first data set => output list of perturbations 
# take a 2nd dataset and get list of perturbations + their probability 


#representation learning (in particular, self-supervised, multi-view, and transfer learning
#https://registry.opendata.aws/tabula-muris/#usageexamples

#see what preprocessing is needed to get better accuracy

##load data

##import premade model => output list of predicted unseen perturbation
#sc.pp.calculate_qc_metrics(adata)

#write custom model. get better accuracy

In [62]:
from collections import defaultdict
rowGeneExpression = defaultdict(int)
rowGeneExpression2 = defaultdict(dict)


import math
math.floor
df = adata.obs
end = 50 if True else -1
for column in hv_columns:
    for row_id in range(math.floor(float(df.shape[0])))[:end]:
        rowGeneExpression[row_id] += adata.X[row_id, column]
        #rowGeneExpression2[row_id][column] = adata.X[row_id, column]


In [63]:
df = adata.obs
dependent_variables =  [rowGeneExpression[row] for row in range(df.shape[0])]
df['geneExpressionCount'] = dependent_variables
numerical_values = df.select_dtypes(include=[int, float]).values.tolist()
dependent_variables = [x for x in dependent_variables]


In [64]:
criterion = torch.nn.CrossEntropyLoss()
#optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [65]:
t_dep = tensor(dependent_variables) # pertrubations
t_dep

tensor([234.6708, 214.1056,  45.8925, 226.1064, 212.4049,  64.8172, 109.4711,  ...,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000], dtype=torch.float64)

In [66]:
t_indep = tensor(numerical_values, dtype=torch.float)
t_indep

tensor([[  3017.0000,    877.0000,    -27.6404,     14.9666,      9.0000,   1704.0000,    711.0000,    234.6708],
        [  2481.0000,    713.0000,    -27.4936,     28.9249,      9.0000,   1614.0000,    662.0000,    214.1056],
        [   703.0000,    337.0000,    -10.4682,     -5.9844,      3.0000,    908.0000,    337.0000,     45.8925],
        [  3420.0000,    850.0000,    -24.3680,     20.4293,      9.0000,   1738.0000,    653.0000,    226.1064],
        [  3158.0000,   1111.0000,     27.9522,     24.1597,      4.0000,   1857.0000,    928.0000,    212.4050],
        [  1869.0000,    635.0000,     -0.4702,    -25.3987,      5.0000,   1525.0000,    634.0000,     64.8172],
        [  1142.0000,    436.0000,    -15.9062,     20.0853,      9.0000,   1157.0000,    436.0000,    109.4711],
        ...,
        [   635.0000,    424.0000,     -6.6479,     -5.5475,      3.0000,    882.0000,    423.0000,      0.0000],
        [  1340.0000,    480.0000,      7.7202,     33.3402,      8.0000,  

In [67]:
vals,indices = t_indep.max(dim=0)
t_indep = t_indep / vals

In [68]:
t_indep = t_indep / vals

In [69]:
independent_variables = pd.DataFrame(numerical_values)

In [97]:
from fastai.data.transforms import RandomSplitter
trn_split,val_split=RandomSplitter(seed=42)(independent_variables)

In [99]:
trn_indep,val_indep = t_indep[trn_split],t_indep[val_split]
trn_dep,val_dep = t_dep[trn_split],t_dep[val_split]
len(trn_indep),len(val_indep)
#dropout, svd, latent space

(19739, 4934)

In [102]:

torch.manual_seed(443)
n_coeff = t_indep.shape[1]

def calc_preds(coeffs, indeps): return (coeffs * indeps).sum(axis=1)
def calc_preds(coeffs, indeps): return torch.sigmoid((indeps*coeffs).sum(axis=1))
#def calc_preds(coeffs, indeps): return (indeps*coeffs).sum(axis=1)
#def calc_preds(coeffs, indeps): return torch.sigmoid(indeps@coeffs)

def calc_loss(coeffs, indeps, deps): return torch.abs(calc_preds(coeffs, indeps)-deps).mean()
def init_coeffs(): return (torch.rand(n_coeff)-0.5).requires_grad_()
def update_coeffs(coeffs, lr): 
    coeffs.sub_(coeffs.grad * lr)
    coeffs.grad.zero_()

def one_epoch(coeffs, lr):
    loss = calc_loss(coeffs, trn_indep, trn_dep)
    loss.backward()
    with torch.no_grad(): update_coeffs(coeffs, lr)
    #print(f"{loss:.3f}", end="; ")
    
# m = torch.nn.Softmax(dim=1)
# t = torch.randn(2, 3)
# one = m(t)
# print(one, t)

def train_model(epochs, lr):
    coeffs = init_coeffs()
    for i in range(epochs):
        one_epoch(coeffs, lr)
    return coeffs

coeffs = train_model(33, lr=.2)

indep_cols = ['nCount_RNA' ,                
'nFeature_RNA'    ,             
'tsne1'     ,                 
'tsne2'  ,                   
'cluster'    ,                  
'nCount_SCT'   ,              
'nFeature_SCT',
'geneExpressionCount'
]    
def show_coeffs(): return dict(zip(indep_cols, coeffs.requires_grad_(False)))
print(show_coeffs())
len([prob for prob in calc_preds(coeffs, t_indep) if prob > .494]) # should be 5000

{'nCount_RNA': tensor(-0.0274), 'nFeature_RNA': tensor(0.4537), 'tsne1': tensor(-0.2658), 'tsne2': tensor(0.3959), 'cluster': tensor(-0.5297), 'nCount_SCT': tensor(-0.0460), 'nFeature_SCT': tensor(-0.4717), 'geneExpressionCount': tensor(-0.2566)}


4662

In [101]:
[prob.item() for prob in calc_preds(coeffs, t_indep)]

[0.48962363600730896,
 0.490382581949234,
 0.4960155487060547,
 0.4898129999637604,
 0.4948292076587677,
 0.49195441603660583,
 0.48960092663764954,
 0.49228110909461975,
 0.4896813631057739,
 0.4880247712135315,
 0.4854820966720581,
 0.4952647387981415,
 0.4926982820034027,
 0.49176910519599915,
 0.494553804397583,
 0.49480435252189636,
 0.4900365173816681,
 0.4848252832889557,
 0.4920617640018463,
 0.49197566509246826,
 0.48953700065612793,
 0.4925662875175476,
 0.4947110712528229,
 0.48512932658195496,
 0.4926314949989319,
 0.4853001832962036,
 0.49218013882637024,
 0.4893205463886261,
 0.49203231930732727,
 0.4951070249080658,
 0.49613630771636963,
 0.4922908842563629,
 0.49017825722694397,
 0.4958970248699188,
 0.4925437867641449,
 0.49203065037727356,
 0.49210691452026367,
 0.49559900164604187,
 0.4953030049800873,
 0.49292200803756714,
 0.4901233911514282,
 0.48938488960266113,
 0.5003027319908142,
 0.48519477248191833,
 0.4949711561203003,
 0.49511051177978516,
 0.4926887750625

In [88]:
coeffs

tensor([-0.0274,  0.4538, -0.2655,  0.3960, -0.4344, -0.0458, -0.4713, -0.2566])

In [87]:
def acc(coeffs): return (val_dep.bool()==(calc_preds(coeffs, val_indep)>0.1)).float().mean()

acc(coeffs) 

tensor(0.0020)

In [80]:
#https://www.10xgenomics.com/resources/datasets/10-k-a-375-cells-transduced-with-1-non-target-and-1-target-sg-rna-dual-indexed-3-1-standard-4-0-0
# !pip install --quiet hyperopt
# !pip install --quiet "ray[tune]"
# !pip install --quiet scvi-colab
# from scvi_colab import install

# install()


# import ray
# import scanpy as sc
# import scvi
# from ray import tune
# from scvi import autotune
# model_cls = scvi.model.SCVI
# model_cls.setup_anndata(adata)
# scvi_tuner = autotune.ModelTuner(model_cls)


In [85]:
def benchmarkResults():
    pass

In [86]:
import scgen
adata.obs.rename({"label": "condition"}, axis=1, inplace=True)
adata.obs["condition"].replace({"ctrl": "control", "stim": "stimulated"}, inplace=True)

#  are very local features enough or do we need global context? How much variation is there and what form does it take? What variation is spurious and could be preprocessed out? Does spatial position matter or do we want to average pool it out? How much does detail matter and how far could we afford to downsample the images? How noisy are the labels?


# https://genomebiology.biomedcentral.com/articles/10.1186/s13059-022-02605-1

# adata_t = adata[
#     ~(
#         (adata.obs["cell_type"] == "CD4 T cells")
#         & (adata.obs["condition"] == "stimulated")
#     )
# ].copy()

# cd4t_stim = adata[
#     (
#         (adata.obs["cell_type"] == "CD4 T cells")
#         & (adata.obs["condition"] == "stimulated")
#     )
# ].copy()








# scgen.SCGEN.setup_anndata(adata_t, batch_key="condition", labels_key="cell_type")


In [83]:
# model = scgen.SCGEN(adata_t, n_hidden=800, n_latent=100, n_layers=2)

In [84]:
# model.train(
#     max_epochs=1, batch_size=32, early_stopping=True, early_stopping_patience=25
# )

In [643]:
# adata_t.obsm["scgen"] = model.get_latent_representation()

In [644]:
# sc.pp.neighbors(adata_t, use_rep="scgen")
# sc.tl.umap(adata)

In [645]:
# adata_t

In [646]:
# pred, delta = model.predict(
#     ctrl_key="control", stim_key="stimulated", celltype_to_predict="CD4 T cells"
# )

# # we annotate the predicted cells to distinguish them later from ground truth cells.
# pred.obs["condition"] = "prediction"
# print(pred)

In [647]:
# pred.obs

In [648]:

# train = adata
# ctrl_adata = adata[
#     ((adata.obs["cell_type"] == "CD4 T cells") & (adata.obs["condition"] == "control"))
# ]
# stim_adata = train[((train.obs['cell_type'] == 'CD4T') & (train.obs['condition'] == 'stimulated'))]

# # concatenate pred, control and real CD4 T cells in to one object
# eval_adata = ctrl_adata.concatenate(cd4t_stim, pred)

In [649]:
# sc.tl.pca(eval_adata)
# sc.pl.pca(eval_adata, color="condition", frameon=False)

In [650]:
# cd4t_adata = adata[adata.obs["cell_type"] == "CD4 T cells"]

In [651]:
# sc.tl.rank_genes_groups(cd4t_adata, groupby="condition", method="wilcoxon")
# diff_genes = cd4t_adata.uns["rank_genes_groups"]["names"]["stimulated"]
# diff_genes

In [ ]:
# r2_value = model.reg_mean_plot(
#     eval_adata,
#     axis_keys={"x": "predicted stimulated", "y": "stimulated"},
#     gene_list=diff_genes[:10],
#     top_100_genes=diff_genes,
#     labels={"x": "predicted", "y": "ground truth"},
#     show=True,
#     legend=False,
# )

In [ ]:
# sc.pl.violin(eval_adata, keys="ISG15", groupby="condition")

In [ ]:
# import pertpy as pt
# import muon as mu
# import scanpy as sc

In [ ]:
# mdata = pt.dt.papalexi_2021()
# for col in mdata.obs: print(col)

In [ ]:
# sc.pp.normalize_total(mdata["rna"])
# sc.pp.log1p(mdata["rna"])
# sc.pp.highly_variable_genes(mdata["rna"], subset=True)

In [ ]:
# mu.prot.pp.clr(mdata["adt"])

In [ ]:
# sc.pp.pca(mdata["rna"])

In [ ]:
# We calculate neighbors with the cosine distance similarly to the original Seurat implementation
# sc.pp.neighbors(mdata["rna"], metric="cosine")

In [ ]:
# sc.tl.umap(mdata["rna"])

In [ ]:
# sc.pl.umap(mdata["rna"], color=["replicate", "Phase", "perturbation"])

In [ ]:
# ms = pt.tl.Mixscape()

# ms.perturbation_signature(
#     mdata["rna"],
#     pert_key="perturbation",
#     control="NT",
#     split_by="replicate",
#     n_neighbors=20,
# )

In [ ]:
# We create a copy of the object to recalculate the PCA.
# Alternatively we could replace the X of the RNA part of our MuData object with the `X_pert` layer.
# adata_pert = mdata["rna"].copy()
# adata_pert.X = adata_pert.layers["X_pert"]
# sc.pp.pca(adata_pert)
# sc.pp.neighbors(adata_pert, metric="cosine")
# sc.tl.umap(adata_pert)
# sc.pl.umap(adata_pert, color=["replicate", "Phase", "perturbation"])

In [ ]:
# ms.mixscape(adata=mdata["rna"], control="NT", labels="gene_target", layer="X_pert")

In [ ]:
# mdata["rna"].obs

In [ ]:
# pt.pl.ms.perturbscore(
#     adata=mdata["rna"], labels="gene_target", target_gene="IFNGR2", color="orange"
# )

In [ ]:
# sc.settings.set_figure_params(figsize=(10, 10))
# pt.pl.ms.violin(
#     adata=mdata["rna"],
#     target_gene_idents=["NT", "IFNGR2 NP", "IFNGR2 KO"],
#     groupby="mixscape_class",
# )

In [ ]:
# pt.pl.ms.heatmap(
#     adata=mdata["rna"],
#     labels="gene_target",
#     target_gene="IFNGR2",
#     layer="X_pert",
#     control="NT",
# )

In [ ]:
# mdata["adt"].obs["mixscape_class_global"] = mdata["rna"].obs["mixscape_class_global"]
# pt.pl.ms.violin(
#     adata=mdata["adt"],
#     target_gene_idents=["NT", "JAK2", "STAT1", "IFNGR1", "IFNGR2", "IRF1"],
#     keys="PDL1",
#     groupby="gene_target",
#     hue="mixscape_class_global",
# )

In [ ]:
# ms.lda(adata=mdata["rna"], labels="gene_target", layer="X_pert")

In [ ]:
# pt.pl.ms.lda(adata=mdata["rna"])
#https://zenodo.org/record/7058382
# folders = '/home/awahab/llm-testing/data_sets/'
# fp1 = 'SC3_v3_NextGem_DI_CRISPR_A549_5K_Multiplex_count_raw_feature_bc_matrix.h5'
# fp2 = 'SC3_v3_NextGem_DI_CRISPR_A549_5K_Multiplex_count_raw_molecule_info.h5'


# import h5py
# import anndata

# # Read the .h5 File
# def explore_h5py_group(group, indent=0):
#     """Recursively print the contents of an h5py group/dataset."""
#     print(group.data.name)
#     items = sorted(group.items())
#     for name, item in items:
#         if isinstance(item, h5py.Dataset):  # Check if item is a dataset
#             print("  " * indent + f"Dataset: {name} (Shape: {item.shape}, Dtype: {item.dtype})")
#         elif isinstance(item, h5py.Group):  # Check if item is a group
#             print("  " * indent + f"Group: {name}")
#             explore_h5py_group(item, indent + 1)  # Recursive call to explore subgroups

# # Open your HDF5 file
# with h5py.File(folders + fp1, 'r') as f:
#     explore_h5py_group(f)
#hv_genes = filtered_keys = df[df['column_name'] == True].index.tolist()

#normal_genes = filtered_keys = df[df['column_name'] == False].index.tolist()
#https://karpathy.github.io/2019/04/25/recipe/
#https://karpathy.medium.com/yes-you-should-understand-backprop-e2f06eab496b
#check bookmarks
#https://www.youtube.com/watch?v=qlLChbHhbg4&ab_channel=AndrejKarpathy
#print(hv_genes, normal_genes)
#https://github.com/karpathy/nn-zero-to-hero

# import numpy as np
# import matplotlib.pyplot as plt
# %matplotlib inline


# plt.hist(hv_columns, bins=30, edgecolor="k")  # `bins` determines the number of bins/bars.
# plt.title("Histogram of Data")
# plt.xlabel("Value")
# plt.ylabel("Frequency")
# plt.show()